# Lead Conversion

## Import Dataset

In [1]:
import pandas as pd

leads = pd.read_csv('Leads.csv')
leads.info()

/home/sourish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/sourish/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
Prospect ID                                      9240 non-null object
Lead Number                                      9240 non-null int64
Lead Origin                                      9240 non-null object
Lead Source                                      9204 non-null object
Do Not Email                                     9240 non-null object
Do Not Call                                      9240 non-null object
Converted                                        9240 non-null int64
TotalVisits                                      9103 non-null float64
Total Time Spent on Website                      9240 non-null int64
Page Views Per Visit                             9103 non-null float64
Last Activity                                    9137 non-null object
Country                                          6779 non-null object
Specialization                                   7802 

### Split X and y

In [2]:
y = leads.iloc[:,6]
X = leads.drop(columns=['Prospect ID', 'Lead Number', 'Converted',
                        'Lead Profile', 'Asymmetrique Activity Index', 'Asymmetrique Profile Index'])
X.head()

,Lead Origin,Lead Source,Do Not Email,Do Not Call,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,...,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,City,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,API,Olark Chat,No,No,0.0,0,0.0,Page Visited on Website,NaN,Select,...,Interested in other courses,Low in Relevance,No,No,Select,15.0,15.0,No,No,Modified
1,API,Organic Search,No,No,5.0,674,2.5,Email Opened,India,Select,...,Ringing,NaN,No,No,Select,15.0,15.0,No,No,Email Opened
2,Landing Page Submission,Direct Traffic,No,No,2.0,1532,2.0,Email Opened,India,Business Administration,...,Will revert after reading the email,Might be,No,No,Mumbai,14.0,20.0,No,Yes,Email Opened
3,Landing Page Submission,Direct Traffic,No,No,1.0,305,1.0,Unreachable,India,Media and Advertising,...,Ringing,Not Sure,No,No,Mumbai,13.0,17.0,No,No,Modified
4,Landing Page Submission,Google,No,No,2.0,1428,1.0,Converted to Lead,India,Select,...,Will revert after reading the email,Might be,No,No,Mumbai,15.0,18.0,No,No,Modified


## Data Preprocessing

### Replace values

In [3]:
import numpy as np

## Replace Select with NaN
X = X.replace(to_replace='Select', value=np.nan)

## Merge Similar Labels
X['Lead Source'] = X['Lead Source'].replace('google', 'Google')
X['Lead Source'] = X['Lead Source'].replace(['blog', 'WeLearn', 'welearnblog_Home'], 'Blog')
X['Country'] = X['Country'].replace(['unknown'], np.nan)

### Display Number of NaN values

In [4]:
X.isnull().sum() * 100 / len(X)

Lead Origin                                       0.000000
Lead Source                                       0.389610
Do Not Email                                      0.000000
Do Not Call                                       0.000000
TotalVisits                                       1.482684
Total Time Spent on Website                       0.000000
Page Views Per Visit                              1.482684
Last Activity                                     1.114719
Country                                          26.688312
Specialization                                   36.580087
How did you hear about X Education               78.463203
What is your current occupation                  29.112554
What matters most to you in choosing a course    29.318182
Search                                            0.000000
Magazine                                          0.000000
Newspaper Article                                 0.000000
X Education Forums                                0.0000

### Encoding Categorical values

In [5]:
print(X.shape)
X = pd.get_dummies(X)
print(X.shape)

(9240, 31)
(9240, 194)


### Fill missing values using KNN

In [6]:
# from fancyimpute import KNN

# X.iloc[:, 0:5] = KNN(k=3).fit_transform(X.iloc[:, 0:5])

In [7]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=np.nan, strategy='mean')
X.iloc[:, 0:5] = imp.fit_transform(X.iloc[:, 0:5])

In [8]:
pd.DataFrame(X).head()

,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score,Lead Origin_API,Lead Origin_Landing Page Submission,Lead Origin_Lead Add Form,Lead Origin_Lead Import,Lead Origin_Quick Add Form,...,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Modified,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked
0,0.0,0.0,0.0,15.0,15.0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,5.0,674.0,2.5,15.0,15.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,1532.0,2.0,14.0,20.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.0,305.0,1.0,13.0,17.0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2.0,1428.0,1.0,15.0,18.0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X.iloc[:, 0:5] = scaler.fit_transform(X.iloc[:, 0:5])
X.head()

,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Asymmetrique Activity Score,Asymmetrique Profile Score,Lead Origin_API,Lead Origin_Landing Page Submission,Lead Origin_Lead Add Form,Lead Origin_Lead Import,Lead Origin_Quick Add Form,...,Last Notable Activity_Form Submitted on Website,Last Notable Activity_Had a Phone Conversation,Last Notable Activity_Modified,Last Notable Activity_Olark Chat Conversation,Last Notable Activity_Page Visited on Website,Last Notable Activity_Resubscribed to emails,Last Notable Activity_SMS Sent,Last Notable Activity_Unreachable,Last Notable Activity_Unsubscribed,Last Notable Activity_View in browser link Clicked
0,0.000000,0.000000,0.000000,0.727273,0.444444,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0.019920,0.296655,0.045455,0.727273,0.444444,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.007968,0.674296,0.036364,0.636364,1.000000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.003984,0.134243,0.018182,0.545455,0.666667,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.007968,0.628521,0.018182,0.727273,0.777778,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## Feature Selection

### Heat Map for correlation among attributes

In [10]:
# temp_X = pd.DataFrame(X)
# temp_y = pd.DataFrame(y)
# temp = pd.concat([temp_X, temp_y], axis=1)

# %matplotlib inline
# import matplotlib.pyplot as plt

# import seaborn as sns
# corrmat = temp.corr()
# top_corr_features = corrmat.index

# ## Plot heat map
# plt.figure(figsize=(90,70))
# g=sns.heatmap(temp[top_corr_features].corr(), annot=True)

### Delete columns with least correlation with "Converted"

In [11]:
# co_mat = temp[top_corr_features].corr()
# un_feat = co_mat['Converted'][co_mat['Converted'] < -0.08]
# indices = un_feat.index.tolist()
# print(indices)

### Feature Importance

In [12]:
# from sklearn.ensemble import ExtraTreesClassifier
# model = ExtraTreesClassifier()
# model.fit(X,y)

# # ## Plot graph of feature importances
# feat_importances = pd.Series(model.feature_importances_)
# # # feat_importances.nsmallest(20).plot(kind='barh')
# # # plt.show()

# indices = feat_importances.nsmallest(20).index
# print(indices)

### Delete column with least feature importance

In [13]:
# X = X.drop(indices, axis=1)
# X = np.delete(X, indices, axis=1)
X.shape

(9240, 194)

## Split Training and Testing Data

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

## Training

In [15]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

In [16]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Testing

In [17]:
y_pred = classifier.predict(X_test)
y_pred

array([1, 1, 0, ..., 0, 1, 1])

### Classification Report

In [18]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.95      0.96      0.96      1143
          1       0.93      0.92      0.93       705

avg / total       0.94      0.94      0.94      1848



In [19]:
y_prob = classifier.predict_proba(X)*100
y_prob[: ,1]

array([ 1.96711238,  2.59185585, 98.36157465, ...,  3.05771108,
       51.80428297, 98.22547321])

In [20]:
classifier.classes_

array([0, 1])

In [21]:
# # Create regularization penalty space
# penalty = ['l1', 'l2']

# # Create regularization hyperparameter space
# C = np.logspace(0, 4, 10)

# # Create hyperparameter options
# hyperparameters = dict(C=C, penalty=penalty)

In [22]:
# from sklearn.model_selection import GridSearchCV

# gs = GridSearchCV(classifier, hyperparameters, cv=5, verbose=0, scoring='accuracy')
# gs.fit(X, y)

In [23]:
np.reshape(np.array(X.columns).T, (-1,1)).shape

(194, 1)

In [24]:
classifier.coef_.T.shape

(194, 1)

In [25]:
# gs.best_params_

In [46]:
feature_importance=pd.DataFrame(np.hstack((np.reshape(np.array(X.columns).T, (-1, 1)), classifier.coef_.T)), columns=['feature', 'importance'])

In [48]:
feature_importance['importance']=pd.to_numeric(feature_importance['importance'])
feature_importance.sort_values(by='importance', ascending=False)

,feature,importance
3,Asymmetrique Activity Score,4.789018
146,Tags_Lost to EINS,4.595798
1,Total Time Spent on Website,3.916118
138,Tags_Closed by Horizzon,3.527994
155,Tags_Will revert after reading the email,2.936588
119,What matters most to you in choosing a course_...,2.023435
24,Lead Source_Welingak Website,1.720344
44,Last Activity_SMS Sent,1.038423
190,Last Notable Activity_SMS Sent,0.897408
162,Lead Quality_High in Relevance,0.764376


In [38]:
feature_importance['importance'][feature_importance['importance'].item() < 1 and feature_importance['importance'].item() > -1]

ValueError: can only convert an array of size 1 to a Python scalar

In [42]:
feature_importance[feature_importance.importance.between(-0.5 , 0.5)]

,feature,importance
0,TotalVisits,0.103877
2,Page Views Per Visit,-0.472210
4,Asymmetrique Profile Score,0.304362
5,Lead Origin_API,-0.189935
8,Lead Origin_Lead Import,-0.295704
9,Lead Origin_Quick Add Form,0.095036
10,Lead Source_Blog,-0.181301
11,Lead Source_Click2call,0.067947
12,Lead Source_Direct Traffic,-0.375961
14,Lead Source_Google,-0.148046
